<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202401/blob/main/Modulo07/Modulo07_PCA_Text_as_Data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Ejemplos Principal Component Analysis

## Test Scores

We are going to use Duflo, Dupas and Kremer (2011) dataset which is a sample of Kenyan first grade test scores.  These authors focus on the variable totalscore which is each student’s composite test score. 

If you examine the data file you will find other pieces of information about the students’ performance, including each student’s score on separate sections of the test, with the labels wordscore (word recognition), sentscore (sentence recognition), letterscore (letter recognition), spellscore (spelling), additions_score (addition), substractions_score (subtraction), multiplications_score (multiplication). 

The “total” score sums the scores from the individual sections. Perhaps there is more information in the section scores. How can we learn about this from the data?

Principal component analysis (PCA) addresses this issue by ordering linear combinations by their contribution to variance.

In [ ]:
require("pacman")
p_load("tidyverse")

In [ ]:
dta<-openxlsx::read.xlsx("https://github.com/ignaciomsarmiento/datasets/raw/main/duflo_data.xlsx")
head(dta)

Variables:

- pupilid        : ID of student
- wordscore      : Endline Score on Word Recognition section (max: 24)
- sentscore      : Endline Score on Sentence Recognition section (max: 40 )
- letterscore    : Endline Score on Letter Recognition section (max: 70)
- spellscore     : Endline Score on Spelling section (max: 10)
- additions_score: Endline Score on Additions section
- substractionscore: Endline Score on Substractions section
- multiplicationscore: Endline Score on Multiplications section


In [ ]:
dta<-dta[,-1]

In [ ]:
cor(dta)

In [ ]:
res_pca <- prcomp(dta, scale=TRUE)
res_pca

In [ ]:
p_load("factoextra")
eig_val <- get_eigenvalue(res_pca)
eig_val

In [ ]:
fviz_eig(res_pca, addlabels = TRUE, ylim = c(0, 70))

In [ ]:
eig_val$pcs<-seq(1:7)
ggplot(eig_val,aes(x=pcs,y=cumulative.variance.percent)) +
    geom_point()+
    geom_line() +
    theme_bw()

In [ ]:
fviz_pca_var(res_pca, 
                repel = TRUE,# Avoid text overlapping
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

In [ ]:
#?fviz_pca_biplot

##  Votos en el congreso de Estados Unidos

Este es un ejemplo un poco mas complejo de como votan los miembros del Congreso de Estados IUnidos. 

Tenemso información de los votos del 111vo Congreso que cubre los años 2009 y 2010, que son los dos primeros años de la presidencia de Obama.

Hay 445 miembros que votan en la casa de representantes (US House of Representatives) y se registraron 1647 votos con -1 por el negativo, +1 por positivo, y 0 por la abstencion o la ausencia

In [ ]:
votes <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-votes.csv")
head(votes)

In [ ]:
dim(votes)

In [ ]:
legis <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-members.csv")
head(legis)

En este contexto podriamos pensar para una estructura factorial subjacente de baja dimensión. Aunque cada uno de los votos se refiere a temas diferentes, los representantes suelen están alineados según ejes partidistas e ideológicos (por ejemplo, republicano versus demócrata o liberal versus conservador). Si creemos que todos los votos son partidistas, entonces el voto por el miembro i en el tema j podría predecirse como

$$
x_{i} = h f_i
$$

donde f es el posicionamiento ideologico latente y h como este posicionamiento afecta los votos. Este tipo de situaciones es ideal para PCA

In [ ]:
#Estimamos PCA
pcavote <- prcomp(votes, scale=TRUE)

pcavote$rotation

In [ ]:
fviz_eig(pcavote, addlabels = TRUE, ylim = c(0, 40))

In [ ]:
fviz_pca_biplot(pcavote,
                labelsize = 20)

In [ ]:
fviz_pca_biplot(pcavote, 
                col.ind = legis$party,
                palette = c("blue", "green", "red"),
                invisible ="var",
                repel=TRUE,
                labelsize = 20
                )

In [ ]:
votepc <- predict(pcavote) #
votepc[order(votepc[,1])[1:5],1]

In [ ]:
votepc[order(-votepc[,1])[1:5],1]

In [ ]:
votepc[order(votepc[,2])[1:5],2]

In [ ]:
votepc[order(-votepc[,2])[1:5],2]

In [ ]:
loadings <- pcavote$rotation[,1:2]

In [ ]:
hist(loadings[,1], main="", xlab="1st Principle Component Vote-Loadings",
     col=8, border=grey(.9))
abline(v=loadings[884,1], col=2)
text(x=loadings[884,1], y=550, "Afford. Health (amdt.)", xpd=TRUE, col=2, font=3)
abline(v=loadings[25,1], col=4)
text(x=loadings[25,1], y=550, "TARP", xpd=TRUE, col=4, font=3)

In [ ]:
loadings[order(abs(loadings[,2]), decreasing=TRUE)[1:5],2]

In [ ]:
sort(rowSums(votes==0), decreasing=TRUE)[1:5]

# Principal Component Regression

Para ilustrar vamos a usar datos de television que incluyen respuestas a encuestas para grupos focales sobre programas piloto de televisión (primeros episodios de una nueva serie), así como los resultados del primer año de calificaciones (cuántas personas terminaron viendo el programa). La esperanza es que podamos crear una regla para predecir el interés de los espectadores a partir de encuestas piloto, ayudando así a los estudios a tomar mejores decisiones de programación. 



In [ ]:
shows <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_showdetails.csv", row.names=1)
shows$Genre <- factor(shows$Genre)
head(shows)

Tenemos un par de variables de resultado interesantes . Las medidas clásicas de la comerciabilidad de la radiodifusión son las calificaciones. Específicamente, los puntos de rating bruto (GRP) proporcionan un recuento estimado de la audiencia total. En estos datos también realizamos un seguimiento del compromiso proyectado (PE) como una medida más sutil de la atención de la audiencia. Después de ver un programa, se contacta a los espectadores y se les pregunta sobre el orden y los detalles de los eventos del programa. Esto mide su compromiso con el programa (y, quizás lo más importante, con los anuncios mostrados). La PE se informa en una escala de 0 a 100, donde 100 significa que estaban completamente comprometidos y 0 significa que no prestaron atención en absoluto.



In [ ]:
plot(GRP ~ PE, data=shows, bg=c(4,2,3)[shows$Genre], pch=21, log="y")
legend("bottomright", legend=levels(shows$Genre), fill=c(4,2,3), bty="n")

Los datos de la encuesta incluyen 6241 vistas y 20 preguntas para 40 programas. Hay dos tipos de preguntas en la encuesta. Ambos  preguntan hasta qué punto estás de acuerdo con una afirmación. Para el primer trimestre, esta afirmación toma la forma de "Este programa me hace sentir...". Para el segundo trimestre, la afirmación es "Encuentro que este programa me hace sentir...". 



In [ ]:
survey <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_pilotsurvey.csv", as.is=TRUE) 
survey$Show <- factor(survey$Show, levels=rownames(shows))
head(survey)

In [ ]:
Xpilot <- aggregate(survey[,-(1:2)],  ## -(1:2) to remove the variables 'show' and 'viewer' completely
                by=list(Show=survey$Show), mean)


rownames(Xpilot) <- Xpilot[,1]
Xpilot <- Xpilot[,-1]
head(Xpilot)

In [ ]:
PCApilot <- prcomp(Xpilot, scale=TRUE)

fviz_eig(PCApilot, addlabels = TRUE, ylim = c(0, 45))

In [ ]:
round(PCApilot$rotation[,1:3],1)

In [ ]:
#options(repr.plot.width = 100, repr.plot.height =100, repr.plot.res = 100)
fviz_pca_biplot(PCApilot, 
                col.ind = shows$Genre,
                palette = c("blue", "green", "red"),
                ylim=c(-3,3),
                xlim=c(-6,6), # hides "monarch cove",living with ed", and "next" but these are all tiny 
                invisible ="var",
                repel=TRUE,
                labelsize = 20
                )

In [ ]:
p_load("gamlr")

In [ ]:
zpilot <- predict(PCApilot)

PE <- shows$PE
zdf <- as.data.frame(zpilot)

summary(PEglm <- glm(PE ~ ., data=zdf[,1:2]))

In [ ]:
cvlasso <- cv.gamlr(x=as.matrix(Xpilot), y=PE, nfold=10)
coef(cvlasso) 

In [ ]:
cvlassoPCR <- cv.gamlr(x=zpilot, y=PE, nfold=10) 
coef(cvlassoPCR) 

In [ ]:
cvlassoboth <- cv.gamlr(x=as.matrix(cbind(Xpilot,zpilot)), y=PE, nfold=10)
coef(cvlassoboth)

In [ ]:
par(mfrow=c(1,3), mai=c(.2,.2,.5,.1), omi=c(.5,.5,0,0))
plot(cvlasso, main="Lasso on X", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoPCR, main="Lasso on PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoboth, main="Lasso on X and PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
mtext(side=2, "mean squared error", outer=TRUE, line=2)
mtext(side=1, "log lamba", outer=TRUE, line=2)

# Texto como Datos: Ejemplos

## Regresión con Texto: Gentzkow and Shapiro

<div >
<img src = "figures/gentzgow_shapiro.png" />
</div>

In [ ]:
require("pacman")
p_load("tidyverse","textir","wordcloud")

In [ ]:
data("congress109", package = "textir")

In [ ]:
congress109Counts[c("Barack Obama","John Boehner"),995:999]

In [ ]:
congress109Ideology[c("Barack Obama","John Boehner"),1:5]

### Regresión 

$$
RepShare= X\beta  + u
$$

In [ ]:
dim(congress109Counts)

In [ ]:
X <- as(congress109Counts, "dMatrix")
repshare <- congress109Ideology$repshare

In [ ]:
X

In [ ]:

X1 <- as.matrix(congress109Counts)
words<-colnames(X1)

covariance<-apply(X1,2,function(x) cor(x,repshare))

                  

wordcloud(words = words,
          freq = covariance,
          min.freq = 0,
          scale = c(1.5, 0.1), 
          max.words=200, 
          random.order=FALSE, 
          colors=brewer.pal(8, "YlOrRd"))


In [ ]:
# lasso
lassoslant <- cv.gamlr(X>0, repshare)  

B <- coef(lassoslant$gamlr)[-1,] 

In [ ]:
tail(sort(round(B[B!=0],4)),10)

In [ ]:
head(sort(round(B[B!=0],4)),10)

## Modelado de temas subyacentes


In [1]:
data("we8there", package = "textir")

Waffle House #1258 in Bossier City, Louisiana:

*I normally would not revue a Waffle House but this one deserves it. The workers, Amanda, Amy, Cherry, James and J.D. were the most pleasant crew I have seen. While it was only lunch, B.L.T. and chili, it was great. The best thing was the 50’s rock and roll music, not to loud not to soft. This is a rare exception to what you all think a Waffle House is. Keep up the good work.*

*Overall: 5, Atmosphere: 5, Food: 5, Service: 5, Value: 5.*



Sartin’s Seafood in Nassau Bay, Texas,

*Had a very rude waitress and the manager wasn’t nice either.*
*Overall: 1, Atmosphere: 1, Food: 1, Service: 1, Value: 5.*

In [7]:
x <- we8thereCounts


In [8]:
x[1,x[1,]!=0]

even though larg portion  mouth water     red sauc    babi back     back rib 
           1            1            1            1            1            1 
chocol mouss veri satisfi 
           1            1

### PCA

In [3]:
pca <- prcomp(x, scale=TRUE) # cuidado demora mucho

In [ ]:
tail(sort(pca$rotation[,1]))

In [ ]:
tail(sort(pca$rotation[,4]))

In [ ]:
v <- predict(pca)[,1:4]

In [ ]:
fviz_eig(pca, addlabels = TRUE, ylim = c(0, 45))

In [ ]:
boxplot(v[,1] ~ we8thereRatings$Overall, xlab="overall rating", ylab="PC1 score")

### LDA

In [ ]:
p_load("maptpx") # para modelar topicos

x <- as.simple_triplet_matrix(we8thereCounts)

In [ ]:
tpc <- topics(x,K=10) 

In [ ]:
tpcs <- topics(x,K=5*(1:5), verb=1)

In [ ]:
summary(tpcs, n=10) 

In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,1], decreasing=TRUE)[1:10]]


In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,2], decreasing=TRUE)[1:10]]

In [ ]:
boxplot(tpcs$omega[,1] ~ we8thereRatings$Overall, col="gold", xlab="overall rating", ylab="topic 1 score")


In [ ]:
boxplot(tpcs$omega[,2] ~ we8thereRatings$Overall, col="pink", xlab="overall rating", ylab="topic 2 score")

In [ ]:
stars <- we8thereRatings[,"Overall"]

In [ ]:
Xtopics<-as(tpcs$omega, "dMatrix")

In [ ]:
p_load("gamlr")
regtopics.cv <- cv.gamlr(tpcs$omega, stars)

In [ ]:
regwords.cv <- cv.gamlr(we8thereCounts, stars)

In [ ]:
par(mfrow=c(1,2), mai=c(.3,.6,.7,.1), omi=c(.5,.2,0,0))
plot(regtopics.cv, ylim=c(1,2), xlab="", ylab="")
mtext("topic regression", font=2, line=2)
plot(regwords.cv, ylim=c(1,2), xlab="", ylab="")
mtext("token regression", font=2, line=2)
mtext(side=2, "mean squared error", outer=TRUE, line=0)
mtext(side=1, "log lamba", outer=TRUE, line=1)